# Acoustic Shield Deployment Pipeline
This notebook demonstrates end-to-end deployment using SageMaker and AWS CLI.

## 1. Launch SageMaker Processing Job (Data Generation)

In [ ]:
!aws sagemaker create-processing-job \
  --profile acm-hack \
  --processing-job-name acousticshield-augment-001 \
  --role-arn <role-sagemaker-processing-arn> \
  --app-specification ImageUri=<your-image-uri>,ContainerEntrypoint=['python3','/opt/ml/processing/augment.py'] \
  --environment "[{"Name":"COUNT_PER_CLASS","Value":"500"}]" \
  --processing-output-config "[{"OutputName":"audio","S3Output":{"S3Uri":"s3://acousticshield-ml/train/","LocalPath":"/opt/ml/outputs/audio","UploadMode":"EndOfJob"}}]"

## 2. Launch SageMaker Training Job (Model Fine-tuning)

In [ ]:
!aws sagemaker create-training-job \
  --profile acm-hack \
  --training-job-name acousticshield-train-001 \
  --role-arn <role-sagemaker-train-arn> \
  --algorithm-specification TrainingImage=<your-image-uri>,TrainingInputMode=File \
  --input-data-config "[{"ChannelName":"train","DataSource":{"S3DataSource":{"S3Uri":"s3://acousticshield-ml/train/","S3DataType":"S3Prefix","S3DataDistributionType":"FullyReplicated"}}}}]" \
  --output-data-config S3OutputPath=s3://acousticshield-ml/model/ \
  --resource-config InstanceType=ml.m5.xlarge,InstanceCount=1,VolumeSizeInGB=30 \
  --hyper-parameters "{"EPOCHS":"3","BATCH_SIZE":"8"}"

## 3. Deploy SageMaker Endpoint (Inference)

In [ ]:
!aws sagemaker create-endpoint-config \
  --profile acm-hack \
  --endpoint-config-name acousticshield-endpoint-config \
  --production-variants "[{"VariantName":"AllTraffic","ModelName":"acousticshield-model","InitialInstanceCount":1,"InstanceType":"ml.m5.xlarge"}]"

In [ ]:
!aws sagemaker create-endpoint \
  --profile acm-hack \
  --endpoint-name acousticshield-endpoint \
  --endpoint-config-name acousticshield-endpoint-config

## 4. Test Endpoint with Sample Audio

In [ ]:
import base64, json, requests
with open('sample.wav', 'rb') as f:
payload = {'audio_b64': audio_b64}
r = requests.post('<API_GATEWAY_URL>/predict', json=payload)
print(r.json())